In [ ]:
import numpy as np
from deap import base, creator, tools, algorithms
from KalmanFilter import KalmanFilter
from SignalGenerator import SignalGenerator
import matplotlib.pyplot as plt
import multiprocessing

# Define the evaluation function
def evaluate(individual):
    q_val, r_val = individual
    F = np.array([[1]])
    H = np.array([[1]])
    Q = np.array([[q_val]])
    R = np.array([[r_val]])
    P = np.array([[0]])
    x = np.array([[0]])

    kf = KalmanFilter(F, H, Q, R, P, x)
    filtered_signal = kf.filter(noisy_signal)
    _, rmse = kf.calculate_error(true_signal, filtered_signal)
    return rmse,

# Kalman Filter parameters
F = np.array([[1]])  # State transition matrix
H = np.array([[1]])  # Observation matrix
Q = np.array([[0]])  # Process noise covariance (to be optimized)
R = np.array([[0]])  # Observation noise covariance (to be optimized)
P = np.array([[0]])  # Initial state covariance
x = np.array([[0]])  # Initial state

# Create a synthetic signal
frequency = 480  # Hz
sampling_rate = 20000  # Hz
duration = 0.1  # seconds (100 ms)
amplitude = 0.1
offset = 0.1

SigGen = SignalGenerator(frequency, sampling_rate, duration, amplitude)
true_signal = SigGen.generate_signal('square')
true_signal = SigGen.add_offset(true_signal, offset)
noise_std = 0.3
noisy_signal = SigGen.add_noise(true_signal, noise_std)


# Genetic Algorithm setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize the RMSE
creator.create("Individual", list, fitness=creator.FitnessMin)  # Create individuals as lists
# Create the toolbox for genetic operations
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 1e-6, 1e3) # Random float for Q and R in the range [1e-6, 1e3]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2) # Each individual has two parameters: Q and R
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # Create the genetic operations

toolbox.register("mate", tools.cxBlend, alpha=0.5) # Blend crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3) # Tournament selection
toolbox.register("evaluate", evaluate) # Evaluation function to calculate RMSE for each individual
# Use elitism to preserve the best individuals
#toolbox.register("select", tools.selBest)
# Enable parallel processing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

# Run the genetic algorithm
population = toolbox.population(n=200) # Population size
ngen = 500  # Number of generations
cxpb = 0.7  # Crossover probability
mutpb = 0.3 # Mutation probability
halloffame = tools.HallOfFame(1)  # Preserve the best individual

# Statistics to keep track of the progress
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Run the genetic algorithm
result, log = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

# Extract the best individual
# best_individual = tools.selBest(population, k=1)[0]
best_individual = halloffame[0]
best_q, best_r = best_individual
print(f"Best Q: {best_q}, Best R: {best_r}")

# Create filter with optimal parameters
optimal_Q = np.array([[best_q]])
optimal_R = np.array([[best_r]])
P = np.array([[0]])
x = np.array([[0]])
optimal_kf = KalmanFilter(F, H, optimal_Q, optimal_R, P, x)
optimal_filtered = optimal_kf.filter(noisy_signal)

# Plot results with optimal parameters
plt.figure(figsize=(12, 6))
plt.plot(noisy_signal, label='Noisy signal')
plt.plot(true_signal, label='True signal')
plt.plot(optimal_filtered, label='Optimized filter')
plt.legend()
plt.title(f'Kalman Filter with Optimal Parameters (Q={best_q:.6f}, R={best_r:.6f})')
plt.show()

# Close the pool
pool.close()
pool.join()